In [5]:
import pandas as pd
from collections import Counter

In [3]:
df = pd.read_json('train-100.json')
df.head()

,path,diagnostic_superclass,scp_labels,diagnostic_one_hot,scp_one_hot
0,../../../../../data/padmalab_external/special_...,[NORM],"[NORM, SR]","[0, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,../../../../../data/padmalab_external/special_...,"[CD, HYP]","[RVH, RAO/RAE, IRBBB, ABQRS, SR]","[1, 1, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,../../../../../data/padmalab_external/special_...,[MI],"[IMI, SR]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,../../../../../data/padmalab_external/special_...,[NORM],"[NORM, SR]","[0, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,../../../../../data/padmalab_external/special_...,[NORM],"[NORM, SR]","[0, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [11]:
len(df)

14204

In [25]:
# Initialize empty DataFrames for each diagnostic class
df_CD = pd.DataFrame(columns=df.columns)
df_HYP = pd.DataFrame(columns=df.columns)
df_MI = pd.DataFrame(columns=df.columns)
df_NORM = pd.DataFrame(columns=df.columns)
df_STTC = pd.DataFrame(columns=df.columns)

# Create a mapping of diagnostic classes to corresponding DataFrames
diagnostic_df_map = {
    'CD': df_CD,
    'HYP': df_HYP,
    'MI': df_MI,
    'NORM': df_NORM,
    'STTC': df_STTC
}

# Iterate over each row in the original DataFrame
for index, row in df.iterrows():
    # Get the list of diagnostic classes for the current row
    diagnostic_classes = row['diagnostic_superclass']
    
    # Iterate over each diagnostic class and add the row to the respective DataFrame
    for diag_class in diagnostic_classes:
        if diag_class in diagnostic_df_map:
            diagnostic_df_map[diag_class] = pd.concat([diagnostic_df_map[diag_class], pd.DataFrame([row])])

# Unpack the DataFrames from the map
df_CD = diagnostic_df_map['CD']
df_HYP = diagnostic_df_map['HYP']
df_MI = diagnostic_df_map['MI']
df_NORM = diagnostic_df_map['NORM']
df_STTC = diagnostic_df_map['STTC']


In [26]:
# Print the DataFrames to see the result
print("CD DataFrame:")
print(len(df_CD))
print("\nHYP DataFrame:")
print(len(df_HYP))
print("\nMI DataFrame:")
print(len(df_MI))
print("\nNORM DataFrame:")
print(len(df_NORM))
print("\nSTTC DataFrame:")
print(len(df_STTC))

CD DataFrame:
3179

HYP DataFrame:
1759

MI DataFrame:
3407

NORM DataFrame:
6592

STTC DataFrame:
3418


In [27]:
# Function to calculate max, min, and mode of SCP codes count
def calculate_scp_statistics(df, class_name):
    scp_counts = df['scp_labels'].apply(len)
    max_count = scp_counts.max()
    min_count = scp_counts.min()
    mode_count = scp_counts.mode()[0] if not scp_counts.mode().empty else None

    print(f"{class_name} DataFrame:")
    print(f"  Maximum number of SCP codes: {max_count}")
    print(f"  Minimum number of SCP codes: {min_count}")
    print(f"  Mode number of SCP codes: {mode_count}\n")

# Calculate and print statistics for each DataFrame
calculate_scp_statistics(df_CD, 'CD')
calculate_scp_statistics(df_HYP, 'HYP')
calculate_scp_statistics(df_MI, 'MI')
calculate_scp_statistics(df_NORM, 'NORM')
calculate_scp_statistics(df_STTC, 'STTC')

CD DataFrame:
  Maximum number of SCP codes: 9
  Minimum number of SCP codes: 1
  Mode number of SCP codes: 3

HYP DataFrame:
  Maximum number of SCP codes: 9
  Minimum number of SCP codes: 1
  Mode number of SCP codes: 3

MI DataFrame:
  Maximum number of SCP codes: 9
  Minimum number of SCP codes: 1
  Mode number of SCP codes: 3

NORM DataFrame:
  Maximum number of SCP codes: 5
  Minimum number of SCP codes: 1
  Mode number of SCP codes: 2

STTC DataFrame:
  Maximum number of SCP codes: 9
  Minimum number of SCP codes: 1
  Mode number of SCP codes: 3



In [29]:
# Function to calculate max, min, and mode of SCP codes count
def calculate_scp_statistics(df, class_name):
    scp_counts = df['scp_labels'].apply(len)
    max_count = scp_counts.max()
    min_count = scp_counts.min()
    mode_count = scp_counts.mode()[0] if not scp_counts.mode().empty else None

    return class_name, max_count, min_count, mode_count

# List to store statistics
statistics = []

# Calculate statistics for each DataFrame and store in list
statistics.append(calculate_scp_statistics(df_CD, 'CD'))
statistics.append(calculate_scp_statistics(df_HYP, 'HYP'))
statistics.append(calculate_scp_statistics(df_MI, 'MI'))
statistics.append(calculate_scp_statistics(df_NORM, 'NORM'))
statistics.append(calculate_scp_statistics(df_STTC, 'STTC'))

# Create a DataFrame from the statistics
stats_df = pd.DataFrame(statistics, columns=['Diagnostic Class', 'Max SCP Codes', 'Min SCP Codes', 'Mode SCP Codes'])

stats_df

,Diagnostic Class,Max SCP Codes,Min SCP Codes,Mode SCP Codes
0,CD,9,1,3
1,HYP,9,1,3
2,MI,9,1,3
3,NORM,5,1,2
4,STTC,9,1,3


In [32]:
# Save the DataFrame to a CSV file without the index column
stats_df.to_csv('scp_codes_activation_stats_lookup.csv', index=False)
